<a href="https://colab.research.google.com/github/kametaro2021a/lab/blob/main/20210728Labo00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プログラミングによる日本語での自然な会話のやり取りを実現したい
## プログラムは **`Python`** を用いる
### 日本語の解析は `mecab` や `SudachiPy` などの便利なライブラリを利用
### その他に **マルコフ連鎖** 用の `markovify` も
### あとは `GTP-2` が無料で利用できる。
### **人口無脳** な `Rinne` などの学習済みデータセットも有用だ。

　さて，人口無脳のプログラムの歴史は古い。
随分以前にはよく遊んだが最近はどうなっているのかとググりながら眺めたが，
新しい言語や手法を用いてはいても基本的に「意識」や「物事を理解する」事がほぼ実装できていないというか，それ以前に解析できていないので，データセットが大きくなったりはしているが基本的なところはあまり変わらないようだ。

　だから先ずは遊んでみるという初学者だとすれば，
基本的な知識や方向性は古の人口無脳と同じで良いだろう。


　具体的には
1. 会話の内容を「理解する」エンジンを作ることは基本的に放棄する。
2. したがって，会話はその都度そこで「用いられた言葉」を元に構成して
あたかも「何事かを理解して返事している風を装う」言葉を紡ぐエンジンを模索する。
3. その為には，こちら（つまり人間側）が発した言葉を「解析する」必要がある。
昔はそこが厄介だったが，
今は `mecab` のような **形態素解析** 用の実績を積んで充実したライブラリがある。
そこで **形態素解析** で得られたデータから，
基本的にはマルコフ連鎖を用いて会話文を捏造する。

とまぁそんなところだ。

現在最先端な `GTP-3` はまだ有料なので今回は使わない。




次のようなプログラムというかコードで必要なライブラリ群を導入しておく。
```
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install markovify
```
ライブラリを導入するだけでは使えないので，
```
import MeCab
import random
```
のようにプログラムでは実際に使うことを宣言しておく必要がある。
あとは，
```
#mecab = MeCab.Tagger("-Ochasen")
mecab = MeCab.Tagger("-Owakati")
print(mecab.parse("すもももももももものうち"))
```
のようにすれば早速使って試すことができる。





最初のライブラリ群の導入は1分弱掛かるか。
結構な量のファイル群が取り込まれる様子が流れる。
そのうち慣れっこになるだろうが最初は驚くと思う。

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install markovify

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

次のライブラリの読み込みは一瞬だ。

In [3]:
import MeCab
import random

そして実際に**形態素解析**を試してみる。
有名な「すもももももももものうち」だ。

In [5]:
#mecab = MeCab.Tagger("-Ochasen")
mecab = MeCab.Tagger("-Owakati")
print(mecab.parse("すもももももももものうち"))

すもも も もも も もも の うち 



一寸だけ時間が掛かるけどほぼ一瞬だ。
所謂この場合は分かち書き（文法的な単語を区切って「英語のように」書く）だ。

ただこれではどの語がどういうものかはさっぱり分からない。
もちろん分からないのは見ている人間なだけで，プログラムはちゃんと識別している。
それは`mecab`のオプションを変えれば見ることができる。

In [6]:
mecab = MeCab.Tagger("-Ochasen")
#mecab = MeCab.Tagger("-Owakati")
print(mecab.parse("すもももももももものうち"))

すもも	スモモ	すもも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
の	ノ	の	助詞-連体化		
うち	ウチ	うち	名詞-非自立-副詞可能		
EOS



上の説明の2つ目のセンテンスをぶちこんでみた。

In [8]:
mecab = MeCab.Tagger("-Owakati")
print(mecab.parse('ただこれではどの語がどういうものかはさっぱり分からない。 もちろん分からないのは見ている人間なだけで，プログラムはちゃんと識別している。 それはmecabのオプションを変えれば見ることができる。'))

ただ これ で は どの 語 が どういう もの か は さっぱり 分から ない 。 もちろん 分から ない の は 見 て いる 人間 な だけ で ， プログラム は ちゃんと 識別 し て いる 。 それ は mecab の オプション を 変えれ ば 見る こと が できる 。 



ちゃんと分かち書きができているような気がする。

さて，このあとは何が必要だろうか。
